In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/Data.csv')

In [39]:
df.drop(labels=['id'], axis=1, inplace=True)

In [40]:
X = df.drop(labels=['price'], axis =1)
Y =df[['price']]

In [41]:
categorical_col = X.select_dtypes(include='object').columns
numerical_col = X.select_dtypes(exclude='object').columns

In [42]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [43]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [44]:
# Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
    ]
)

# Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoding', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
    ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_col),
    ('cat_pipeline', cat_pipeline, categorical_col)
])


In [45]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.30, random_state=42)    

In [48]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.fit_transform(X_test), columns=preprocessor.get_feature_names_out())

In [50]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [51]:
regression = LinearRegression()
regression.fit(X_train, Y_train)

LinearRegression()

In [52]:
regression.coef_

array([[ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
         -494.17005097,   -76.32351645,    68.80035873,  -464.67990411,
          652.10059539]])

In [53]:
regression.intercept_

array([3976.8787389])

In [55]:
def evaluate_model(true_value, predicted_value):
    r2 = r2_score(true_value, predicted_value)
    mae = mean_absolute_error(true_value, predicted_value)
    mse = mean_squared_error(true_value, predicted_value)
    rmse = np.square(mean_squared_error(true_value, predicted_value))
    return r2, mae, mse, rmse

In [58]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    r2,mae, mse,rmse=evaluate_model(Y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2*100)

    r2_list.append(r2)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1061720402724.6257
MAE: 678.6599280042763
R2 score 93.62334489637519


Lasso
Model Training Performance
RMSE: 1061639455777.8636
MAE: 679.9362394438181
R2 score 93.62358798325504


Ridge
Model Training Performance
RMSE: 1061729857012.7942
MAE: 678.693463514639
R2 score 93.62331650537858


Elasticnet
Model Training Performance
RMSE: 5425346217927.448
MAE: 1065.7688041796291
R2 score 85.5854338619943


